In [ ]:
import numpy as np
import keras
import tensorflow as tf
import pandas as pd

In [ ]:
data = pd.read_excel("C:\Requerimientos.xlsx", sheetname='Hoja1')


print('Número de datos Cargados: {num}'.format(num=data.shape[0]))
data = data.dropna()
# Imprimimos el número de tweets a procesar
print('Número de datos a procesar: {num}'.format(num=data.shape[0]))
data.sample(10)

In [ ]:
data.dtypes


In [ ]:
data.groupby('REQU_TIRE_ID')['REQU_DETALLE'].count().reset_index().sort_values(by='REQU_DETALLE', ascending=False)


In [ ]:
data1 = data[data["REQU_TIRE_ID"] != -1]
data1.groupby('REQU_TIRE_ID')['REQU_DETALLE'].count().reset_index().sort_values(by='REQU_DETALLE', ascending=False)

In [ ]:

data2 = data1.groupby('REQU_TIRE_ID')['REQU_DETALLE'].count().reset_index().sort_values(by='REQU_DETALLE', ascending=False)

data2

data3 = data2.loc[data2['REQU_DETALLE'] < 100].REQU_TIRE_ID.iloc[:].values
data1 = data1[~data1.REQU_TIRE_ID.isin(data3)]

data1.groupby('REQU_TIRE_ID')['REQU_DETALLE'].count()

In [ ]:
coment = [tuple(x) for x in data1[['REQU_TIRE_ID', 'REQU_DETALLE']].values]
print('Número de comentarios Cargados: {num}'.format(num=len(coment)))
# Carga de librerías

In [ ]:
print(coment[8])

In [ ]:
import re
from unicodedata import normalize
texto = 'gasto medicaménto permanente'

# -> NFD y eliminar diacríticos
s1 = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( 'NFD', X_norm[33]), 0, re.I
    )

# -> NFC
s1 = normalize( 'NFC', s1)

print( s1 )

In [ ]:
txt= 'Hola! Soy Ben Asistente Virtual de tu Seguro Integral tratare de ayudarte en lo que pueda.Que Te Importa \xa0\xa0Hola Por favor ingresa tu RUT sin puntos y con guion ejemplo 12345678-9Que Te Importa \xa0\xa018155047-3 Ingresa tu emailQue Te Importa \xa0\xa0bernarditaarias...'
print(txt)
#ignore = 
txt2 = txt.lower().replace('.','. ').replace('\xa0','. ').replace('-','. ').strip()
t1 = str.maketrans(dict.fromkeys('0123456789',""))
txt2 = txt2.translate(t1)

sentencia = sent_tokenize(txt2)
ignore_sen =  ['hola!','soy ben asistente virtual de tu seguro integral tratare de ayudarte en lo que pueda.', 'hola por favor ingresa tu rut sin puntos y con guion ejemplo .']

txtfin = ''.join([grama for grama in sentencia if grama not in ignore_sen])# 'Soy Ben Asistente Virtual de tu Seguro Integral tratare de ayudarte en lo que pueda.'])
print(txtfin)

In [ ]:
import numpy as np
import spacy
import string
import re
from unicodedata import normalize
import spacy_spanish_lemmatizer
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

nlp = spacy.load("es_core_news_sm")
nlp.replace_pipe("lemmatizer", "spanish_lemmatizer")

from tqdm import tqdm
en_stop = set(stopwords.words('spanish'))
ignore_frase =  ['hola!', 'soy ben asistente virtual de tu seguro integral tratare de ayudarte en lo que pueda.', 'hola por favor ingresa tu rut sin puntos y con guion ejemplo .']
ignore_words = ['estimados','estimadas','estimado','estimada','hola','favor','asegurado','asistente','tratare','juntos','siempre','ayudarte','virtual', 'indicar', 'procedimiento', 'digital', 'buenos','dias','rut', 'estimados','ayuda','bancochile','selecciona','ben', 'maritza','cl','rut','puntos','guion','email','ejemplo','sistema', 'nombre']
e_palabra_caracter_intermedio = ["@","°", "º"]
e_palabra_caracter_inicial = ["http", "https", "º", "°"]
e_palabra_caracter_final = [".com",".cl",".org","º", "°"]


#nlp = spacy.load('es_core_news_sm')

# Divido los datos en dos listas 
#     X: los tweets
#     y: target (polaridad)

X = [doc[1] for doc in coment]
y = np.array([doc[0] for doc in coment])

def normalizar(sentenses):
    #print(sentenses)
    """normalizamos la lista de frases y devolvemos la misma lista de frases normalizada"""
    for index, sentense in enumerate(tqdm(sentenses)):
        lema = ''
        #print(sentense)
        # Tokenizamos el tweets realizando los puntos 1,2 y 3.
        if type(sentense) != int:
            #print(sentense)
            s = re.sub(
                    r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                    normalize( 'NFD', sentense), 0, re.I
                )
            #print(sentense)
            sentense = normalize( 'NFC', s)
            regex = re.compile("|".join(f"\S+{cad}\S+" for cad in e_palabra_caracter_intermedio))
            sentense = regex.sub("", sentense)
            regex = re.compile("|".join(f"{cad}\S+" for cad in e_palabra_caracter_inicial))
            sentense = regex.sub("", sentense)
            regex = re.compile('|'.join(f"\S+{cad}" for cad in e_palabra_caracter_final))
            sentense = regex.sub("", sentense).replace("  ", " ")
            t = str.maketrans(dict.fromkeys('0123456789',""))
            sentense = sentense.lower().replace('.','. ').replace('\xa0','. ').replace('-','. ').strip()
            #print(sentense)
            sentense = sentense.translate(t)
            #print(sentense)
            sentense = sent_tokenize(sentense)
            sentense = ''.join([fra for fra in sentense if fra not in ignore_frase])
            sentense = sentense.lower().replace('.',' ').replace('-','').replace('"','').replace('º','').replace('°','').strip()
            sentense = sentense.translate(t)
            word_tokens = word_tokenize(sentense)
            #print(tokenizer.tokenize(sentense[0]))
            #print(sentense)
            # Puntos 4,5 y 6
            sentense = ' '.join([word for word in word_tokens if word not in ignore_words 
                                        and word not in en_stop
                                        and word not in string.punctuation])
            #word_grama = word_tokenize(sentense)
            #sentenses[index] = list(nltk.ngrams(word_grama,2))
            #print(sentense)
            for token in nlp(sentense):
                lema = lema + ' ' + token.lemma_
            #print(lema)
            sentenses[index] = lema
            #print(sentenses[index])
        #(not word.is_punct)
         #                    and (not word.text.startswith('@'))
          #                   and (not word.text.startswith('http'))
           #                  and (not ':' in word.text)])
    return sentenses

# Normalizamos las frases


In [ ]:
from sklearn.model_selection import train_test_split
seed = 7
validation_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X_norm,y,test_size=validation_size, shuffle=True, random_state=seed)
print('comentarios de entrenamiento: {}'.format(len(X_train)))
print('comentarios de Test: {}'.format(len(X_test)))

In [ ]:
#X_train.astype(int)
np.array(X_train, dtype=np.str)

In [ ]:
PCT_TEST = 0.25
n_tail = len(X_norm) - int(len(X_norm) * PCT_TEST)
print('Corte en el comentario número {} de los {} tweets del Dataset.'.format(n_tail, len(X_norm)))

X_train = X_norm[:n_tail]
y_train = y[:n_tail]
X_test = X_norm[n_tail:]
y_test = y[n_tail:]

print('comentarios de entrenamiento: {}'.format(len(X_train)))
print('comentarios de Test: {}'.format(len(X_test)))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import one_hot

# Hacemos un one-hot encoder del texto
VOCAB_SIZE = 100000 # Poner un valor muy alto
X_train = [one_hot(doc, VOCAB_SIZE) for doc in np.array(X_train, dtype=np.str)]
X_test = [one_hot(doc, VOCAB_SIZE) for doc in np.array(X_test, dtype=np.str)]

# Codificación del Target
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
#encoder.fit(y_test)
y_test = encoder.transform(y_test)


In [ ]:
#Creación de secuencia de palabras
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_WORDS_COMENT = 32
pad_corpus_train = pad_sequences(X_train, maxlen=MAX_WORDS_COMENT, padding='post')
pad_corpus_test = pad_sequences(X_test, maxlen=MAX_WORDS_COMENT, padding='post')
print(pad_corpus_train)

In [ ]:
print(y_test)
import keras.backend as K
K.clear_session()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization, GRU, Bidirectional
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import adam
from keras.layers import Embedding

EMBEDDING_SIZE = 
# Set Model
model1 = Sequential()
model1.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE))
model1.add(Bidirectional(GRU(EMBEDDING_SIZE)))
#model1.add(LSTM(128, return_sequences=True))
model1.add(Dropout(0.2))
model1.add(BatchNormalization())

# Set Optimizer
opt = adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)

# Compile model
model1.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)
model1.summary()

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, LSTM, GRU, SimpleRNN, Bidirectional, BatchNormalization
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


EMBEDDING_SIZE = 32
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE))
#model.add(GRU(EMBEDDING_SIZE, return_sequences=True))
model.add(Bidirectional(GRU(EMBEDDING_SIZE)))
model.add(Dropout(0.2))
#model.add(GRU(32))
#model.add(Embedding(100000, EMBEDDING_SIZE))
#model.add(GRU(EMBEDDING_SIZE))
model.add(BatchNormalization())
model.add(Dense(16, activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer=opt)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Imprimimos la arquitectura de la red
model.summary()

In [ ]:
import datetime
import os

from tensorflow.keras.callbacks import TensorBoard

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs('tensorboard_logs/{}'.format(now))
path = os.path.join('tensorboard_logs', now)

history=model.fit(pad_corpus_train, 
                  y_train, 
                  batch_size=16, 
                  epochs=10, 
                  validation_data=(pad_corpus_test, y_test), 
                  verbose=2,
                  callbacks=[TensorBoard(log_dir=path,
                                         histogram_freq=2,
                                         write_graph=True,
                                         write_images=True)])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Pintamos las métricas por epoch
def plot_metric(history, name, remove_first=0):
    metric_train = np.array(history.history[name])[remove_first:]
    metric_test = np.array(history.history['val_{}'.format(name)])[remove_first:]
    acum_avg_metric_train = (np.cumsum(metric_train) / (np.arange(metric_train.shape[-1]) + 1))[remove_first:]
    acum_avg_metric_test = (np.cumsum(metric_test) / (np.arange(metric_test.shape[-1]) + 1))[remove_first:]
    plt.figure(figsize=(20, 8))
    plt.subplot(1, 2, 1)
    plt.title('{} - Epochs'.format(name))
    plt.plot(metric_train, label='{} Train'.format(name))
    plt.plot(metric_test, label='{} Test'.format(name))
    plt.grid()
    plt.legend(loc='upper center')
    plt.subplot(1, 2, 2)
    plt.title('AVG ACCUMULATIVE {} - Epochs'.format(name))
    plt.plot(acum_avg_metric_train, label='{} Train'.format(name))
    plt.plot(acum_avg_metric_test, label='{} Test'.format(name))
    plt.grid()
    plt.legend(loc='upper center')
    plt.show()

In [ ]:
# Función de perdida
plot_metric(history=history, name='loss')

In [ ]:
# Accuracy
plot_metric(history=history, name='acc')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

y_true = encoder.inverse_transform(y_test.reshape(-1))
y_pred = model.predict_classes(pad_corpus_test)
y_pred = encoder.inverse_transform(y_pred.reshape(-1))

print('Accuracy: {acc:0.4f}'.format(acc=accuracy_score(y_true=y_true, y_pred=y_pred)))
print('F1: {f1:0.4f}'.format(f1=f1_score(y_true=y_true, y_pred=y_pred, average='weighted')))
print('Precision: {pre:0.4f}'.format(pre=precision_score(y_true=y_true, y_pred=y_pred, average='weighted')))
print('Recall: {rec:0.4f}'.format(rec=recall_score(y_true=y_true, y_pred=y_pred, average='weighted')))
print(classification_report(y_true=y_true, y_pred=y_pred))
labels = [1001,1005,2000,2036,2046,2051,2058,2072,2075,2124,2129,4029,4030,4035,4039]
confusion_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=labels)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

def plot_confusion_matrix(cm, classes, title, cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'), horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Pintamos la matriz de confusión
plt.figure(figsize=(12, 6))
plot_confusion_matrix(confusion_matrix, classes=labels, title='Matriz de Confusión')
plt.show()

In [ ]:
model.save('model_lstm_32_16_lr_0001_32_16_lematiza')